In [46]:
from torch_geometric.datasets import Planetoid

import numpy as np
import random
from random import shuffle
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from qiskit import *

import torch.optim as optim
from torch.nn.parameter import Parameter
from sklearn.metrics import f1_score
import scipy.sparse as sp
import numpy as np
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.nn import GCNConv
import sys
from givens_roatation import GivensRotations
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchquantum as tq
#import torchquantum.layers
#from tq.layers import Op1QAllLayer,Op2QAllLayer
from measure import Measure
from qlayers import QLayer_block1,QLayer_block2,QLayer_block3
import os
import matplotlib.pyplot as plt  
import seaborn as sns  

In [401]:
dataset = Planetoid(root='./data', name='Cora') # 将数据保存在data文件夹下
data = dataset[0]

In [402]:
# data = data.subgraph(torch.tensor(range(1024)))
labels = data.y
print(max(labels))
for i in range(int(max(labels))+1):
    print( 'label = ',i,'sum =',(labels==i).sum())

tensor(6)
label =  0 sum = tensor(351)
label =  1 sum = tensor(217)
label =  2 sum = tensor(418)
label =  3 sum = tensor(818)
label =  4 sum = tensor(426)
label =  5 sum = tensor(298)
label =  6 sum = tensor(180)


In [403]:
n_node = 512
n_train = 256
n_feature = 512
n_vqc_wires = int(np.log2(n_feature))
label0 = 2
label1 = 3

In [404]:
index34 = (labels==label0)+(labels==label1)
data = data.subgraph(index34)
labels = data.y

In [405]:
shuffle_index = np.array(range(labels.shape[0]))
print(shuffle_index)
#shuffle(shuffle_index)
#print(shuffle_index[0:64])

[   0    1    2 ... 1233 1234 1235]


In [406]:
data = data.subgraph(torch.tensor(shuffle_index[0:n_node]))

In [407]:
features = data.x
features = torch.nn.functional.normalize(features, p=2, dim=0)
print('features original shape:',features.shape)

features original shape: torch.Size([512, 1433])


In [408]:
features_sum = (features*features).sum(0)
non_zero_feature = features_sum>0.01
features = data.x[:,non_zero_feature]
print('features sub shape:',features.shape)

features sub shape: torch.Size([512, 1247])


In [409]:
zeros = torch.zeros(n_node,2048-features.shape[1])
features = torch.hstack([features,zeros])
features = features.reshape(n_node,n_feature,-1).sum(-1).squeeze()
print('features sub shape:',features.shape)


features sub shape: torch.Size([512, 512])


In [410]:
labels = data.y#.to(torch.float)
index1 = labels==label0
index6 = labels==label1
labels[index1]=0
labels[index6]=1
print('labels:',labels)

labels: tensor([1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 

In [411]:
data.x = features
print(features.shape)
data.y = labels
print(data.y)

torch.Size([512, 512])
tensor([1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 

In [412]:
train_index = torch.zeros_like(data.train_mask)
train_index[0:n_train] = 1
train_index = train_index.to(torch.bool)
test_index = torch.zeros_like(data.test_mask)
test_index[n_train:n_node] =1
test_index = test_index.to(torch.bool)
print(train_index.sum(),test_index.sum())
print(train_index.shape)
print(test_index.shape)

tensor(256) tensor(256)
torch.Size([512])
torch.Size([512])


In [413]:
nclass = labels.max().item()+1
edges = data.edge_index.T


print('edge.shape:',edges.shape)
pq_series = []

for i in range(edges.shape[0]):
    # if edges[i][0].item() in range(n_train,n_node) and edges[i][1].item() in range(n_train,n_node):
    #     continue
    if edges[i][0].item() < edges[i][1].item():
        pq_series.append([edges[i][0].item(), edges[i][1].item()])


edge_index = torch.tensor(pq_series).T
# data.edge_index = edge_index
for i in range(int(max(labels))+1):
    print( 'label = ',i,'sum =',(labels==i).sum())

print('edge.shape:',edge_index.shape)

# sys.exit(0)
print(nclass)
print(features.shape)
print(labels.shape)

edge.shape: torch.Size([650, 2])
label =  0 sum = tensor(175)
label =  1 sum = tensor(337)
edge.shape: torch.Size([2, 325])
2
torch.Size([512, 512])
torch.Size([512])


In [414]:
def normalize_adj(mx):

    mx = mx + sp.eye(mx.shape[0])
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1/2).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    mx = mx.dot(r_mat_inv)
    return mx

In [415]:
class QGCN(tq.QuantumModule):
    def __init__(self,g,n_nodes,n_dimension,pqs) :
        super().__init__()
        self.n_qubit =math.ceil(math.log(n_nodes,2))
        self.qgcn1 = GivensRotations(self.n_qubit,pqs)
        # self.qgcn1.genTrans()
        # self.qgcn2 = GivensRotations(self.n_qubit,pqs)
        self.linear1 = torch.nn.Linear(int(np.log2(n_feature)),2)
        # self.linear2 = torch.nn.Linear(32,2)
        self.adj = to_scipy_sparse_matrix(g.edge_index,num_nodes=n_nodes)
        self.adj = normalize_adj(self.adj)
        self.adj =torch.tensor(self.adj.todense(),dtype= torch.float32)
        self.n_wires = n_vqc_wires
        self.q_layer1 = QLayer_block3(self.n_wires)
        self.q_layer2 = QLayer_block3(self.n_wires)
        self.q_layer3 = QLayer_block3(self.n_wires)
        self.q_layer4 = QLayer_block3(self.n_wires)
        self.q_layer5 = QLayer_block3(self.n_wires)
        self.q_layer6 = QLayer_block3(self.n_wires)
        self.q_layer7 = QLayer_block3(self.n_wires)
        self.q_layer8 = QLayer_block3(self.n_wires)
        self.q_layer9 = QLayer_block3(self.n_wires)
        self.q_layer10 = QLayer_block3(self.n_wires)
        self.q_layer11 = QLayer_block3(self.n_wires)
        self.q_layer12 = QLayer_block3(self.n_wires)
        self.q_layer13 = QLayer_block3(self.n_wires)

        self.encoder = tq.StateEncoder()
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        self.measure = Measure(obs=tq.PauliZ,wires= range(self.n_wires))
        # self.n_dimension = n_dimension
    def compare(self):
        adj =self.adj
        givens = self.qgcn1.get_trans()
        return adj, givens

    def forward(self,h):
        bsz = h.shape[0]
        h = self.qgcn1(h)
        #h = torch.mm(self.adj,h)
        #h = h*h
        h = self.encoder(self.q_device,h)
        self.q_layer1(self.q_device)
        #self.q_layer2(self.q_device)
        #self.q_layer3(self.q_device)
        #self.q_layer4(self.q_device)
        #self.q_layer5(self.q_device)
        #self.q_layer6(self.q_device)
        #self.q_layer7(self.q_device)
        #self.q_layer8(self.q_device)
        #self.q_layer9(self.q_device)
        #self.q_layer10(self.q_device)
        #self.q_layer11(self.q_device)
        h = self.measure(self.q_device)
        #print(h.size())
        #print(h)
        h = self.linear1(h)
        #h = h.reshape(bsz, 2, -1).sum(-1).squeeze()
        return h

    def quantum_inference(self,input):
        pass
    # def show_circuit(self):
    #     print(self.qgcn1.circuit)

In [416]:
loss_func = torch.nn.CrossEntropyLoss()


def train(model, data, train_index, lr=0.01, weight_decay=5e-4, epochs=20):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    labels = data.y
    train_mask = train_index

    best_loss_val = 100
    

    for i in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(data.x)
        # print(output.shape)
        loss = loss_func(output[train_mask], labels[train_mask]) # 用训练集中的节点来计算损失函数
        # print(loss)

        loss.backward()
        optimizer.step()
        scheduler.step()
        if i % 10 == 0:
            # print('parameters:',list(model.parameters())[1].item())
            print('Epoch {}, training loss: {}'.format(i, loss.item()))

@torch.no_grad()
def test(model, data,test_index):
    """Evaluate GAT performance on test set.

    """
    model.eval()
    test_mask = test_index
    labels = data.y 
    output = model(data.x) # 得到模型输出

    loss_test = loss_func(output[test_mask], labels[test_mask])
    preds = output[test_mask].argmax(1) # 得到预测值
   
    # print(preds)
    # preds = preds.argmax(1)
    # preds = preds > 0.5
    # print(labels[test_mask])
    # labels = labels > 0.5
    # print(preds) 
    # print(labels[test_mask])
    # sys.exit(0)
    acc_test = preds.eq(labels[test_mask]).cpu().numpy().mean() # 得到准确率
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test))
    return preds, output, acc_test.item()

In [417]:
model = QGCN(data,n_nodes=n_node,n_dimension=1433,pqs=edge_index.T)


device = 'cpu'
model = model.to(device)
data = data.to(device)
train(model, data,train_index=train_index, epochs=50)

num: 325
Epoch 0, training loss: 0.7149457335472107
Epoch 10, training loss: 0.67525315284729
Epoch 20, training loss: 0.6540732383728027
Epoch 30, training loss: 0.6444830894470215
Epoch 40, training loss: 0.6408944129943848


In [418]:
preds, output, acc = test(model, data,test_index=test_index)

Test set results: loss= 0.6373 accuracy= 0.6641
